<a href="https://colab.research.google.com/github/kabilan942/Hugging-Face/blob/main/Amazon%20Review%20Dataset/Hugging_Face_Summarization_en%26es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 42.5 MB/s 
     |████████████████████████████████| 120 kB 54.0 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Download the English and Spanish subsets from the Hugging Face Hub.


In [ ]:
from datasets import load_dataset

english_dataset = load_dataset("amazon_reviews_multi", "en")
spanish_dataset = load_dataset("amazon_reviews_multi", "es")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
english_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [ ]:
english_dataset['train'].features

{'review_id': Value(dtype='string', id=None),
 'product_id': Value(dtype='string', id=None),
 'reviewer_id': Value(dtype='string', id=None),
 'stars': Value(dtype='int32', id=None),
 'review_body': Value(dtype='string', id=None),
 'review_title': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'product_category': Value(dtype='string', id=None)}

In [ ]:
english_dataset['train'][5]

{'review_id': 'en_0420650',
 'product_id': 'product_en_0705898',
 'reviewer_id': 'reviewer_en_0155342',
 'stars': 1,
 'review_body': 'Poor quality. The material was fuzzy from day one. It got discolored in less than a month, even though we kept it outdoors under a covered porch. I returned this item.',
 'review_title': 'Poor quality. The material was fuzzy from day one',
 'language': 'en',
 'product_category': 'industrial_supplies'}

Since each review is accompanied by a short title, we can use the titles as the target summaries for our model to learn from.

Therefore, the review information we are interested in is contained in the **review_body** and review_title columns.

We'll look at a few examples by creating a simple function that takes a random sample from the training set.

We can create a random sample by chaining the **Dataset.shuffle()** and **Dataset.select()** functions together.

The **seed** (= 42 here) is fixed in Dataset.shuffle() for reproducibility purposes.

Dataset.select() expects an iterable of indices, so we’ve passed **range(num_samples)** to grab the first 3 examples from the shuffled dataset.


In [ ]:
def show_samples(dataset, num_samples=3, seed=42):
  sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))   
  # dataset['train'] used as Dataset for shuffle() and dataset["train"].shuffle(seed=seed) is used as Dataset for select()
  for example in sample:
    print()
    print(">> Title: {}".format(example['review_title']))
    print(">> Review: {}".format(example['review_body']))

show_samples(english_dataset)   # num_samples = 3 and seed = 42


>> Title: Worked in front position, not rear
>> Review: 3 stars because these are not rear brakes as stated in the item description. At least the mount adapter only worked on the front fork of the bike that I got it for.

>> Title: meh
>> Review: Does it’s job and it’s gorgeous but mine is falling apart, I had to basically put it together again with hot glue

>> Title: Can't beat these for the money
>> Review: Bought this for handling miscellaneous aircraft parts and hanger "stuff" that I needed to organize; it really fit the bill. The unit arrived quickly, was well packaged and arrived intact (always a good sign). There are five wall mounts-- three on the top and two on the bottom. I wanted to mount it on the wall, so all I had to do was to remove the top two layers of plastic drawers, as well as the bottom corner drawers, place it when I wanted and mark it; I then used some of the new plastic screw in wall anchors (the 50 pound variety) and it easily mounted to the wall. Some have r

When seed is changed from 42 -> 40:

In [ ]:
def show_samples(dataset, num_samples=3, seed=40):
  sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))   
  # dataset['train'] used as Dataset for shuffle() and dataset["train"].shuffle(seed=seed) is used as Dataset for select()
  for example in sample:
    print()
    print(">> Title: {}".format(example['review_title']))
    print(">> Review: {}".format(example['review_body']))

show_samples(english_dataset)   # num_samples = 3 and seed = 42


>> Title: Cheap Screen Protectors
>> Review: Protect my phones screen and easy to install.

>> Title: Broke
>> Review: It broke after 2 months

>> Title: do not like feel of cover
>> Review: returning, do not like feel of cover.


We'll focus on generating summaries for a single domain of products, since training the model on all 400,000 reviews (en+es) will take too long.

To choose the domain, convert english_dataset to Pandas.DataFrame

In [ ]:
english_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [ ]:
english_dataset.set_format('pandas')

In [ ]:
english_dataset['train'][:].head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,en_0964290,product_en_0740675,reviewer_en_0342986,1,Arrived broken. Manufacturer defect. Two of th...,I'll spend twice the amount of time boxing up ...,en,furniture
1,en_0690095,product_en_0440378,reviewer_en_0133349,1,the cabinet dot were all detached from backing...,Not use able,en,home_improvement
2,en_0311558,product_en_0399702,reviewer_en_0152034,1,I received my first order of this product and ...,The product is junk.,en,home
3,en_0044972,product_en_0444063,reviewer_en_0656967,1,This product is a piece of shit. Do not buy. D...,Fucking waste of money,en,wireless
4,en_0784379,product_en_0139353,reviewer_en_0757638,1,went through 3 in one day doesn't fit correct ...,bubble,en,pc


In [ ]:
english_df = english_dataset['train'][:]
english_df.shape

(200000, 8)

In [ ]:
english_df['product_category'].value_counts().shape

(31,)

Let's print the **top 20** products from the dataframe.

In [ ]:
english_df['product_category'].value_counts()[:20]

home                      17679
apparel                   15951
wireless                  15717
other                     13418
beauty                    12091
drugstore                 11730
kitchen                   10382
toy                        8745
sports                     8277
automotive                 7506
lawn_and_garden            7327
home_improvement           7136
pet_products               7082
digital_ebook_purchase     6749
pc                         6401
electronics                6186
office_product             5521
shoes                      5197
grocery                    4730
book                       3756
Name: product_category, dtype: int64

Switching the format of english_dataset from "pandas" back to "arrow":

In [ ]:
english_dataset.reset_format()

Let's focus on summarizing book reviews; the categories that comes under book review are **book** and **digital_ebook_purchase**

In [ ]:
def filter_books(example):   # example -> dataset which was called in the call 
  return (example['product_category']=='book' or example['product_category']=='digital_ebook_purchase' )
    
# filter_books is the function which will be passed as the function in the filter()

In [ ]:
spanish_books = spanish_dataset.filter(filter_books)
english_books = english_dataset.filter(filter_books)

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

As a sanity check:

In [ ]:
show_samples(english_books)


>> Title: Cheap
>> Review: Great info from a history buff

>> Title: Love the idea but...
>> Review: I bought this book so that I could introduce my kids to "A bug and a wish" which is a very similiar teaching to what my mentor teacher taught me once: "I don't like it when you__, can you please___?" which is a nicer way for kids to independently solve their problems. Anyways, if I had seen the pages in this book, I would not have got it. The pages have a lot of unnecessary extra words which do not quite relate to the heart of the story. The setup time of the boy getting bullied was cliched and the ending was unclear. Spoiler: It was his friend that said the lines and then suddenly on the last page, the bullies and the boy were friends. He did not say it to the bullies; she did! And she wasn't the one getting bullied. Great idea but poorly executed. Also the pictures are very pre-k level despite the story being too wordy for a pre-k kid to sit through.

>> Title: Rather boring designs.

In [ ]:
show_samples(spanish_books)


>> Title: Edad recomendada 5 o 6 años
>> Review: El libro está bien, pero aunque se indique que es para niños y niñas de entre 6 y 10 años, en realidad es para 5 o 6. Un par de frases por página, un desarrollo muy sencillo. Lo he tenido que devolver porque es demasiado básico para mi sobrina de 7.

>> Title: Un poco cutre
>> Review: El libro llegó rápido y en perfectas condiciones. No obstante, la edición es muy pobre y el interior es una simple impresión en hoja blanca. No cuenta con índice ni títulos de los capítulos. Si esperas un libro con una edición medianamente elaborada, no compres este ejemplar.

>> Title: Mal
>> Review: Envió muy mal. Después de pasar el plazo de entrega te dicen que en breve lo recibirás. Pero no te dicen cuando


In [ ]:
english_books

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 10505
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 231
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 278
    })
})

In [ ]:
spanish_books

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 7107
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 193
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 164
    })
})

Notice english_books and spanish_books ahve **10505** and **7107** samples from the books domain in the training set out of the 200,000 samples in each initially. 

Using the **concatenate_datasets()** function, we'll stack two Dataset objects (english_books and spanish_books) on top of each other into a single **DatasetDict** object

So, to create our bilingual dataset, we’ll loop over each split, concatenate the datasets for that split, and shuffle the result to ensure our model doesn’t overfit to a single language:

In [ ]:
print(english_books.keys())

dict_keys(['train', 'validation', 'test'])


In [ ]:
english_books['train']

Dataset({
    features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 10505
})

In [ ]:
from datasets import concatenate_datasets, DatasetDict

books_dataset = DatasetDict()  # DatasetDict object where we'll concatenate the english_books and spanish_books datasets

for split in english_books.keys(): # english_books.keys() returns ['train', 'validation', 'test']
  books_dataset[split] = concatenate_datasets([english_books[split], spanish_books[split]])
  # the above code concatenates 'train' datasets of english_books and spanish_books into 'train' of books_dataset, then does the same for 'validation' and 'test'
  books_dataset[split] = books_dataset[split].shuffle(seed = 42)

In [ ]:
books_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 17612
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 424
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 442
    })
})

Note that num_rows in 'train' of books_dataset = 17612 = 10505 + 7107

In [ ]:
show_samples(books_dataset)


>> Title: Recomendable cien por cien.
>> Review: Es la primera vez que leo algo de Mata Torres y me ha sorprendido gratamente. Libro muy recomendable y entretenido.

>> Title: Un poco rollo
>> Review: Es muy denso de leer. Lo he empezado varias veces y no consigo enterarme de lo que dice. Creí que iba a ser más ameno.

>> Title: Three Stars
>> Review: I've read better from IA. There's so much explanation in the books, you lose the characters.


We’ll **filter out the examples with very short titles** so that our model can produce more interesting summaries. Since we’re dealing with English and Spanish texts, we can use a rough heuristic to split the titles on whitespace and then use Dataset.filter() method







In [ ]:
books_dataset = books_dataset.filter(lambda x: (len(x['review_title'].split())>2))

# the above filter filters out the samples from the dataset which has less than 2 words in its title (words seperated by a white space)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
books_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 9672
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 238
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 245
    })
})

Notice once filtered, that the number of samples is down to **9672** from **17612** in the training set of books_dataset.

## **Preprocessing the data**

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"  # which is a multilingual model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

ValueError: ignored

In [ ]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

In [ ]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'])

The special Unicode character ▁ and end-of-sequence token </s> indicate that we’re dealing with the SentencePiece tokenizer, which is based on the **Unigram segmentation algorithm**.

It is especially useful for multilingual corpora since it allows SentencePiece to be agnostic about accents, punctuation, and the fact that many languages, like Japanese, do not have whitespace characters.

We apply truncation to both the reviews and their titles to ensure we don’t pass excessively long inputs to our model.

The tokenizer has a **as_target_tokenizer()** function that allows you to tokenize the labels in parallel to the inputs.

This is typically done using a context manager inside a preprocessing function that first encodes the inputs, and then encodes the labels as a separate column.

In [ ]:
# Values for max_input_length and max_target_length, which set the upper limits for how long our reviews and titles can be
# Since the review body is typically much larger than the title, the values are scaled accordingly

max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
  # Reviews are first tokenized
  model_inputs = tokenizer(examples['review_body'], truncation=True, max_length=max_input_length)
  # tokenizer for the target (Title)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['review_title'], truncation=True, max_length=max_target_length)
  # The inputs are first encoded, and then the labels ('review_title') are encoded as a separate column
  model_inputs['labels'] = labels['input_ids']
  return model_inputs



Now we can tokenize the whole corpus using the Dataset.map() function with the preprocess_function()

In [ ]:
tokenized_dataset = books_dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_dataset